In [1]:
%matplotlib inline
import numpy as np
from scipy.optimize import root_scalar, minimize_scalar
from math import log, ceil
import ipywidgets as widgets
import matplotlib.pyplot as plt

class StabilityPredictor:
    
    def __init__(self, I, E, N, P, R, g, a, h, rI, rE, rY):      
        self.I=I
        self.E=E
        self.N=N
        self.P=P
        self.R=R
        self.g=g
        self.a=a
        self.h=h
        self.rI=rI
        self.rE=rE
        self.rY=rY
    
    def run(self):
        if not self.check_ss():
            return np.inf, 'FAIL_SS', False
        elif not self.check_initial_condition():
            return np.inf, 'FAIL_IC', False
        current_domain = self.determine_domain();
        time_in_danger = 0
        exceeds_P = False
        current_t = 0
        # Placeholder values to begin while loop
        done = False 
        while not done:
            #Used to calculate new E and I values at crossover point
            temp_E = self.E
            temp_I = self.I
            if current_domain == 'mentee':
                t1 = self.find_mentee_yfr_crossover()
                t2 = self.find_mentee_mentor_crossover()
                t = min((i for i in (t1,t2) if i>0), default=-1)
                if t == -1:
                    if not exceeds_P and self.violates_P_end('mentee', current_t):
                        exceeds_P = True
                    done = True
                else:
                    self.E = self.E_mentee_lim(temp_E, temp_I, t)
                    self.I = self.I_mentee_lim(temp_E, temp_I, t)
                    #Check for voilation of P if crossover point found
                    if not exceeds_P and self.violates_P_mentee(temp_I, temp_E, current_t, current_t+t):
                        exceeds_P = True
                    current_t += t
                    if t == t1:
                        current_domain = 'yfr'
                    else:
                        current_domain = 'mentor'
                        if self.check_under_crit_value():
                            done = True
                
            elif current_domain == 'mentor':
                t1 = self.find_mentor_yfr_crossover()
                t2 = self.find_mentor_mentee_crossover()
                t = min((i for i in (t1,t2) if i > 0), default=-1)
                if t == -1:
                    if not exceeds_P and self.violates_P_end('mentee', current_t):
                        exceeds_P = True
                    done = True
                else:
                    self.E = self.E_mentor_lim(temp_E, temp_I, t)
                    self.I = self.I_mentor_lim(temp_E, temp_I, t)
                    if not exceeds_P and self.violates_P_mentor(temp_I, temp_E, current_t, current_t+t):
                        exceeds_P = True
                    current_t += t
                    time_in_danger += t
                    if t == t1:
                        current_domain = 'yfr'
                    else:
                        current_domain = 'mentee'
            else:
                t1 = self.find_yfr_mentee_crossover()
                t2 = self.find_yfr_mentor_crossover()
                t = min((i for i in (t1,t2) if i > 0), default=-1)
                if t==-1:
                    if not exceeds_P and self.violates_P_end('mentee', current_t):
                        exceeds_P = True
                    done = True
                else:
                    self.E = self.E_yfr_lim(temp_E, t)
                    self.I = self.I_yfr_lim(temp_I, t)
                    if not exceeds_P and self.violates_P_yfr(temp_I, temp_E, current_t, current_t+t):
                        exceeds_P = True
                    current_t += t
                    time_in_danger += t
                    if t == t1:
                        current_domain = 'mentee' 
                    else:
                        current_domain = 'mentor'
                        if self.check_under_crit_value():
                            done = True
        
        if current_domain == 'mentee':
            return time_in_danger, 'PASS', exceeds_P
        elif current_domain == 'yfr':
            return np.inf, 'FAIL_YFR', exceeds_P
        else:
            return np.inf, 'FAIL_MENTOR', exceeds_P
        
                
    def check_ss(self):
        try:
            Iss = self.g*self.R/self.rI
            Ess = (self.g-self.h-self.a*self.N)/self.a
            return Iss>0 and Ess>0 and Iss+Ess<self.P and (self.rE*Ess>self.rI*Iss or self.rE*Ess>self.rY) and\
                (self.rY>self.rI*Iss or self.rY>self.rE*Ess)
        except:
            return False

    def check_initial_condition(self):
        return self.rE*self.E>self.rI*self.I or self.rE*self.E>self.rY or self.E>((self.h+self.a*self.N)/(self.rE/self.R-self.a))
        
    def determine_domain(self):
        current_min = min(self.rI*self.I, self.rE*self.E, self.rY)
        if self.rI*self.I == current_min:
            return 'mentee'
        elif self.rE*self.E == current_min:
            return 'mentor'
        else:
            return 'yfr'
    

    def find_yfr_mentee_crossover(self):
        try:
            return (self.rY/self.rI-self.I)*(self.R/(self.g-self.rY))
        except:
            return -1

    def find_yfr_mentor_crossover(self):
        try:
            k=-self.rY+self.h*self.R+self.a*self.N*self.R
            log_arg = (self.rY*self.a*self.R+self.rE*self.k)/(self.rE*(self.E*self.a*self.R+k))
            return -log(log_arg)/self.a
        except:
            return -1

    def find_mentee_yfr_crossover(self):
        try:
            k1=self.I-self.g*self.R/self.rI
            log_arg = (self.rY-self.g*self.R)/(self.rI*self.k1)
            return -self.R*log(log_arg)/self.rI
        except:
            return -1

    def find_mentee_mentor_crossover(self):
        try:
            k1=self.I-self.g*self.R/self.rI
            k2=self.E+(self.rI/(self.rI-self.a*self.R))*k1-(self.g-self.h-self.a*self.N)/self.a
            func = lambda t: self.rI*(k1*np.exp(-self.rI*t/self.R)+self.g*self.R/self.rI)-\
                self.rE*(-self.rI/(self.rI-self.a*self.R)*k1*np.exp(-self.rI*t/self.R)+k2*np.exp(-self.a*t)+\
                (self.g-self.h-self.a*self.N)/self.a)
            t_sol = root_scalar(func, bracket=[0.01, self.get_bracket_upper('mentee')])
            return t_sol.root
        except:
            return -1

    def find_mentor_yfr_crossover(self):
        k1=self.E-(self.h+self.a*self.N)/(self.rE/self.R-self.a)
        k2=self.I+k1/(1-self.a*self.R/self.rE)
        try:
            log_arg = (self.rY-(self.rE*(self.h+self.a*self.N))/(self.rE/self.R-self.a))/(self.rE*k1)
            return log(log_arg)/(self.rE/self.R-self.a)
        except:
            return -1

    def find_mentor_mentee_crossover(self):
        try:
            k1=self.E-(self.h+self.a*self.N)/(self.rE/self.R-self.a)
            k2=self.I+k1/(1-self.a*self.R/self.rE)
            func = lambda t: self.rE*(k1*np.exp((self.rE/self.R-self.a)*t)+(self.h+self.a*self.N)/(self.rE/self.R-self.a)) -\
                self.rI*(-k1/(1-self.a*self.R/self.rE)*np.exp((self.rE/self.R-self.a)*t)+\
                ((-self.h-self.a*self.N)/(1-self.a*self.R/self.rE)+self.g)*t+k2)
            t_sol = root_scalar(func, bracket=[0.01, self.get_bracket_upper('mentor')])
            return t_sol.root
        except:
            return -1
    
    def get_bracket_upper(self, domain):
        if domain == 'mentee':
            k1=self.I-self.g*self.R/self.rI
            k2=self.E+(self.rI/(self.rI-self.a*self.R))*k1-(self.g-self.h-self.a*self.N)/self.a
            func = lambda t: self.rI*(k1*np.exp(-self.rI*t/self.R)+self.g*self.R/self.rI)-\
                self.rE*(-self.rI/(self.rI-self.a*self.R)*k1*np.exp(-self.rI*t/self.R)+k2*np.exp(-self.a*t)+\
                (self.g-self.h-self.a*self.N)/self.a)
        else:
            k1=self.E-(self.h+self.a*self.N)/(self.rE/self.R-self.a)
            k2=self.I+k1/(1-self.a*self.R/self.rE)
            func = lambda t: self.rE*(k1*np.exp((self.rE/self.R-self.a)*t)+(self.h+self.a*self.N)/(self.rE/self.R-self.a)) -\
                self.rI*(-k1/(1-self.a*self.R/self.rE)*np.exp((self.rE/self.R-self.a)*t)+\
                ((-self.h-self.a*self.N)/(1-self.a*self.R/self.rE)+self.g)*t+k2)
        return self.find_sign_change(func)
        
    def find_sign_change(self, func):
        i = 1
        done = False
        # Checking for sign change within next 40 years
        while i < 2080 and done == False:
            if func(i) > 0:
                done = True
            i+=1
        return i
    
    def check_under_crit_value(self):
        return self.E < (self.h+self.a*self.N)/(self.rE/self.R-self.a)
    
    def E_mentor_lim(self, E, I, t):
        k1=E-(self.h+self.a*self.N)/(self.rE/self.R-self.a)
        k2=I+k1/(1-self.a*self.R/self.rE)
        return k1*np.exp((self.rE/self.R-self.a)*t)+(self.h+self.a*self.N)/(self.rE/self.R-self.a)
    
    def I_mentor_lim(self, E, I, t):
        k1=E-(self.h+self.a*self.N)/(self.rE/self.R-self.a)
        k2=I+k1/(1-self.a*self.R/self.rE)
        return -k1/(1-self.a*self.R/self.rE)*np.exp((self.rE/self.R-self.a)*t)+\
                ((-self.h-self.a*self.N)/(1-self.a*self.R/self.rE)+self.g)*t+k2
    
    def E_mentee_lim(self, E, I, t):
        k1=I-self.g*self.R/self.rI
        k2=E+(self.rI/(self.rI-self.a*self.R))*k1-(self.g-self.h-self.a*self.N)/self.a
        return -self.rI/(self.rI-self.a*self.R)*k1*np.exp(-self.rI*t/self.R)+k2*np.exp(-self.a*t)+\
                (self.g-self.h-self.a*self.N)/self.a
    
    def I_mentee_lim(self, E, I, t):
        k1=I-self.g*self.R/self.rI
        k2=E+(self.rI/(self.rI-self.a*self.R))*k1-(self.g-self.h-self.a*self.N)/self.a
        return k1*np.exp(-self.rI*t/self.R)+self.g*self.R/self.rI
    
    def E_yfr_lim(self, E, t):
        k=-self.rY+self.h*self.R+self.a*self.N*self.R
        return ((E*self.a*self.R+k)*np.exp(-self.a*t)-k)/(self.a*self.R)
    
    def I_yfr_lim(self, I, t):
        return I+(self.g-self.rY/self.R)*t

    def violates_P_mentee(self, I, E, lower, upper):
        k1=I-self.g*self.R/self.rI
        k2=E+(self.rI/(self.rI-self.a*self.R))*k1-(self.g-self.h-self.a*self.N)/self.a
        func = lambda t: self.P - (k1*np.exp(-self.rI*t/self.R)+self.g*self.R/self.rI)-\
                (-self.rI/(self.rI-self.a*self.R)*k1*np.exp(-self.rI*t/self.R)+k2*np.exp(-self.a*t)+\
                (self.g-self.h-self.a*self.N)/self.a)
        minimum = minimize_scalar(func, bounds=(lower,upper)).fun
        return minimum < 0;
    
    def violates_P_mentor(self, I, E, lower, upper):
        k1=E-(self.h+self.a*self.N)/(self.rE/self.R-self.a)
        k2=I+k1/(1-self.a*self.R/self.rE)
        func = lambda t: self.P - (k1*np.exp((self.rE/self.R-self.a)*t)+(self.h+self.a*self.N)/(self.rE/self.R-self.a)) -\
                (-k1/(1-self.a*self.R/self.rE)*np.exp((self.rE/self.R-self.a)*t)+\
                ((-self.h-self.a*self.N)/(1-self.a*self.R/self.rE)+self.g)*t+k2)
        minimum = minimize_scalar(func, bounds=(lower,upper)).fun
        return minimum < 0;

    def violates_P_yfr(self, I, E, lower, upper):
        k=-self.rY+self.h*self.R+self.a*self.N*self.R
        func = lambda t: self.P - (I+(self.g-self.rY/self.R)*t) -\
                (((E*self.a*self.R+k)*np.exp(-self.a*t)-k)/(self.a*self.R))
        minimum = minimize_scalar(func, bounds=(lower,upper)).fun
        return minimum < 0;

    def violates_P_end(self, domain, start):
        func = None
        if domain == 'mentee':
            k1=self.I-self.g*self.R/self.rI
            k2=self.E+(self.rI/(self.rI-self.a*self.R))*k1-(self.g-self.h-self.a*self.N)/self.a
            func = lambda t: self.P - (k1*np.exp(-self.rI*t/self.R)+self.g*self.R/self.rI)-\
                (-self.rI/(self.rI-self.a*self.R)*k1*np.exp(-self.rI*t/self.R)+k2*np.exp(-self.a*t)+\
                (self.g-self.h-self.a*self.N)/self.a)
        elif domain == 'mentor':
            k1=self.E-(self.h+self.a*self.N)/(self.rE/self.R-self.a)
            k2=self.I+k1/(1-self.a*self.R/self.rE)
            func = lambda t: self.P - (k1*np.exp((self.rE/self.R-self.a)*t)+(self.h+self.a*self.N)/(self.rE/self.R-self.a)) -\
                (-k1/(1-self.a*self.R/self.rE)*np.exp((self.rE/self.R-self.a)*t)+\
                ((-self.h-self.a*self.N)/(1-self.a*self.R/self.rE)+self.g)*t+k2)
        else:
            k=-self.rY+self.h*self.R+self.a*self.N*self.R
            func = lambda t: self.P - (self.I+(self.g-self.rY/self.R)*t) -\
                (((self.E*self.a*self.R+k)*np.exp(-self.a*t)-k)/(self.a*self.R))
        
        i = start
        violates = False
        # Checking for sign change within next 40 years
        while i < start+2080 and violates == False:
            if func(i) < 0:
                violates = True
            i+=1
        return violates
    

In [2]:
def calc_u(I, E, R, rI, rE, rY):
    current_min = min(rI*I, rE*E, rY)
    if rI*I == current_min:
        return rI/R*I
    elif rE*E == current_min:
        return rE/R*E
    else:
        return rY/R

def get_vector_coordinates(I, E, N, R, g, a, h, rI, rE, rY):
    u = calc_u(I, E, R, rI, rE, rY)
    x = g-u
    y = u-h-a*E-a*N
    return x, y

# PARSimple Analytic Tool
Welcome to the PARSimple "What If?" Analytic Tool. This tool allows you to modify the parameters to PARSimple and observe several outputs, such as areas of system sustainability, areas of transient voilation of PML and times in danger zones.

### Important Metrics Based on Initial Experienced and Inexperienced Pilots

In [3]:
%matplotlib inline
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap, Normalize

def plot_predictions(N=25, P=30, R=250, g=4, a=0.07,
                     h=1, rI=144, rE=144, rY=10000):
    
    plt.rcParams["figure.figsize"] = (8,5)

    color_vals = np.zeros([P+1,P+1])
    for e in range(0, P+1):
        for i in range(0, P+1):
            predictor = StabilityPredictor(i, e, N, P, R, g/52, a/52, h/52, rI/52, rE/52, rY/52)
            _, _, exceeds_P = predictor.run()
            c_value = None
            if not exceeds_P:
                c_value = 1
            else:
                c_value = 2
            color_vals[e,i] = c_value
            if i == P-e:
                break

    values = [0,1,2]
    cmp = ListedColormap(['white','limegreen', 'lightslategray'])
    im = plt.imshow(color_vals, origin='lower', cmap=cmp, vmin=0, vmax=2)

    colors = [ im.cmap(value) for value in values]
    
    xP = np.array(P)
    dashed_line = plt.axline((0,P), slope=-1, linestyle='dashed', label='E+I=P', c='black')
    patches = [ mpatches.Patch(color=colors[1], label="Not Exceeds P"),
            mpatches.Patch(color=colors[2], label="Exceeds P"),
              dashed_line]
    plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    triangle = plt.Polygon([[-0.5,P+0.5],[P,P],[P+0.5,-0.5]], color='white')
    plt.gca().add_patch(triangle)
    plt.title('Predicted Stability of the System Depending on \nInitial Experienced and Inexperienced Pilots')
    plt.xlabel('I0')
    plt.ylabel('E0')

    # Minor ticks
    plt.xticks(np.arange(-.5, 30.5, 1), minor=True)
    plt.yticks(np.arange(-.5, 30.5, 1), minor=True)
    
    # Gridlines based on minor ticks
    plt.grid(which='minor')

def plot_predictions(N=25, P=30, R=250, g=4, a=0.07,
                     h=1, rI=144, rE=144, rY=10000):
                    
    #fig, axis = plt.subplots(1,2, figsize=(15,5))
    fig, axis = plt.subplots(2,2, figsize=(20,20))
    
    x,y = np.meshgrid(np.linspace(0,P+1,P+1),np.linspace(0,P+1,P+1)) 

    u = np.zeros(x.shape) #x comp, dI
    v = np.zeros(y.shape)
    
    #plt.rcParams["figure.figsize"] = (8,5)
    times = np.empty([P+1,P+1])
    color_vals = np.zeros([P+1,P+1])
    color_vals_2 = np.zeros([P+1,P+1])
    for e in range(0, P+1):
        for i in range(0, P+1):
            x_val, y_val = get_vector_coordinates(i, e, N, R, g/52, a/52, h/52, rI/52, rE/52, rY/52)
            u[e,i] = x_val
            v[e,i] = y_val
            
            predictor = StabilityPredictor(i, e, N, P, R, g/52, a/52, h/52, rI/52, rE/52, rY/52)
            time_in_danger, status, exceeds_P = predictor.run()

            if time_in_danger == np.inf:
                time_in_danger = 500
            times[e,i] = time_in_danger
            
            c_value = None
            if status == 'PASS':
                c_value = 1
            elif status == 'FAIL_SS':
                c_value = 2
            elif status == 'FAIL_IC':
                c_value = 3
            else:
                c_value = 4
            color_vals[e,i] = c_value

            c_value_2 = None
            if status != 'PASS':
                c_value_2 = 3
            else:
                if not exceeds_P:
                    c_value_2 = 1
                else:
                    c_value_2 = 2
            color_vals_2[e,i] = c_value_2
                
            if i == P-e:
                break

    try:
        iss = g*R/rI
        ess = (g-h-a*N)/a
    except:
        iss = 0
        ess= 0
    #color_vals[ess,iss] = 5
    axis[0,0].plot(iss,ess,'ko') 
    axis[0,1].plot(iss,ess,'ko')
    axis[1,0].plot(iss,ess,'ko')
    axis[1,1].plot(iss,ess,'ko')
    
    values = [0,1,2,3,4,5]
    cmp = ListedColormap(['white','limegreen', 'lightslategray', 'chocolate', 'royalblue', 'black'])
    im = axis[0,0].imshow(color_vals, origin='lower', cmap=cmp, vmin=0, vmax=5)

    colors = [ im.cmap(value) for value in values]
    
    xP = np.array(P)
    #dashed_line = plt.axline((0,P), slope=-1, linestyle='dashed', label='E+I=P', c='black')
    dashed_line = axis[0,0].axline((0,P), slope=-1, linestyle='dashed', label='E+I=P', c='black')
    patches = [ mpatches.Patch(color=colors[1], label="Success"),
            mpatches.Patch(color=colors[2], label="Steady State Failure"),
            mpatches.Patch(color=colors[3], label="Initial Condition Failure"),
            mpatches.Patch(color=colors[4], label="Mentor-Limited Crash"),
            mpatches.Patch(color=colors[5], label="Steady State"),
              dashed_line]
    #plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    #subs[3][1].legend(bbox_to_anchor=(2, -.2), ncol=len(target_names))
    axis[0,0].legend(handles=patches, loc='upper right')
    #triangle = plt.Polygon([[-0.5,P+0.5],[P,P],[P+0.5,-0.5]], color='white')
    #plt.gca().add_patch(triangle)
    triangle = plt.Polygon([[-0.5,P+0.5],[P+0.5,P+0.5],[P+0.5,-0.5]], color='white')
    axis[0,0].add_patch(triangle)
    axis[0,0].set_xticks(np.arange(-.5, P+0.5, 1), minor=True)
    axis[0,0].set_yticks(np.arange(-.5, P+0.5, 1), minor=True)
    
    # Gridlines based on minor ticks
    axis[0,0].grid(which='minor')

    axis[0,0].set_title('Stability Based on Initial Experienced and Inexperienced Pilots')
    axis[0,0].set_xlabel('I0')
    axis[0,0].set_ylabel('E0')

    axis[0,1].legend(handles=patches, loc='upper right')
    axis[0,1].quiver(x,y,u,v, color_vals, cmap=cmp, norm=Normalize(vmin=0, vmax=5))
    axis[0,1].set_title('Change in Pilot Populations')
    axis[0,1].set_xlabel('I0')
    axis[0,1].set_ylabel('E0')

    values = [0,1,2,3]
    cmp = ListedColormap(['white','forestgreen', 'tomato', 'saddlebrown'])
    im = axis[1,0].imshow(color_vals_2, origin='lower', cmap=cmp, vmin=0, vmax=3)

    colors = [ im.cmap(value) for value in values]
    
    xP = np.array(P)
    axis[1,0].axline((0,P), slope=-1, linestyle='dashed', label='E+I=P', c='black')
    patches = [ mpatches.Patch(color=colors[1], label="Does Not Exceed P"),
            mpatches.Patch(color=colors[2], label="Exceeds P"),
               mpatches.Patch(color=colors[3], label="Failure"),
               mpatches.Patch(color='black', label="Steady State"),
              dashed_line]
    axis[1,0].legend(handles=patches, loc='upper right')
    triangle2 = plt.Polygon([[-0.5,P+0.5],[P,P],[P+0.5,-0.5]], color='white')
    axis[1,0].add_patch(triangle2)
    axis[1,0].set_title('Transient Violation of P')
    axis[1,0].set_xlabel('I0')
    axis[1,0].set_ylabel('E0')

    # Minor ticks
    axis[1,0].set_xticks(np.arange(-.5, 30.5, 1), minor=True)
    axis[1,0].set_yticks(np.arange(-.5, 30.5, 1), minor=True)
    
    # Gridlines based on minor ticks
    axis[1,0].grid(which='minor')
    
    danger_hm = axis[1,1].imshow(times, cmap='tab10', interpolation='spline16', origin='lower')
    fig.colorbar(danger_hm, ax=axis[1,1], fraction=0.046, pad=0.04)
    axis[1,1].axline((0,P), slope=-1, linestyle='dashed', label='E+I=P', c='black')
    triangle1 = plt.Polygon([[-0.5,P+0.5],[P+0.5,P+0.5],[P+0.5,-0.5]], color='white')
    axis[1,1].add_patch(triangle1)
    axis[1,1].set_xticks(np.arange(-.5, P+0.5, 1), minor=True)
    axis[1,1].set_yticks(np.arange(-.5, P+0.5, 1), minor=True)
    axis[1,1].grid(which='minor')
    axis[1,1].set_title('Times in Danger Zones \n(in weeks, 500 indicates a failed instance)')
    axis[1,1].set_xlabel('I0')
    axis[1,1].set_ylabel('E0')

widgets.interact_manual(plot_predictions, N=widgets.IntSlider(value=25, min=0, max=100),
               P=widgets.IntSlider(value=30, min=0, max=100),
               R=widgets.IntSlider(value=250, min=0, max=1000),
               g=widgets.IntSlider(value=4, min=0, max=100),
               a=widgets.FloatSlider(value=0.07, min=0, max=1, step=0.01),
               h=widgets.IntSlider(value=1, min=0, max=100),
               rI=widgets.IntSlider(value=144, min=0, max=1000),
               rE=widgets.IntSlider(value=144, min=0, max=1000),
               rY=widgets.IntSlider(value=10000, min=0, max=100000))
pass

interactive(children=(IntSlider(value=25, description='N'), IntSlider(value=30, description='P'), IntSlider(va…

### Stability Based on Initial Experienced Pilots and Attrition Rate

In [33]:
%matplotlib inline
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap

def plot_predictions(i=10, N=25, P=30, R=250, g=4,
                     h=1, rI=144, rE=144, rY=10000):
    
    plt.rcParams["figure.figsize"] = (8,3)

    color_vals = np.zeros([P-i+1,31], dtype=int)
    for a in range(0,31):
        for e in range(0, P-i+1):
            predictor = StabilityPredictor(i, e, N, P, R, g/52, a/100/52, h/52, rI/52, rE/52, rY/52)
            _, status, _ = predictor.run()
            c_value = None
            if status == 'PASS':
                c_value = 1
            elif status == 'FAIL_SS':
                c_value = 2
            elif status == 'FAIL_IC':
                c_value = 3
            else:
                c_value = 4
            color_vals[e,a] = c_value

    values = [0,1,2,3,4]
    cmp = ListedColormap(['white','limegreen', 'lightslategray', 'chocolate', 'royalblue'])
    im = plt.imshow(color_vals, origin='lower', cmap=cmp, vmin=0, vmax=4)
    colors = [ cmp(value) for value in values]

    xP = np.array(P)
    patches = [ mpatches.Patch(color=colors[1], label="Success"),
            mpatches.Patch(color=colors[2], label="Steady State Failure"),
            mpatches.Patch(color=colors[3], label="Initial Condition Failure"),
            mpatches.Patch(color=colors[4], label="Mentor-Limited Crash")]
    plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.title('Stability Based on Initial Experienced Pilots and Attrition Rate')
    plt.xlabel('a')
    plt.ylabel('E0')
    plt.xticks(np.arange(-.5, P+0.5, 1), minor=True)
    plt.yticks(np.arange(-.5, P-i+0.5, 1), minor=True)
    plt.grid(which='minor')

widgets.interact_manual(plot_predictions, N=widgets.IntSlider(value=25, min=0, max=100),
               P=widgets.IntSlider(value=30, min=0, max=100),
               R=widgets.IntSlider(value=250, min=0, max=1000),
               g=widgets.IntSlider(value=4, min=0, max=100),
               h=widgets.IntSlider(value=1, min=0, max=100),
               rI=widgets.IntSlider(value=144, min=0, max=1000),
               rE=widgets.IntSlider(value=144, min=0, max=1000),
               rY=widgets.IntSlider(value=10000, min=0, max=100000),
               I=widgets.IntSlider(value=10, min=0, max=100))
pass

interactive(children=(IntSlider(value=10, description='i', max=30, min=-10), IntSlider(value=25, description='…

### Stability Based on Initial Inexperienced Pilots and Attrition Rate

In [107]:
%matplotlib inline
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap

def plot_predictions(e=10, N=25, P=30, R=250, g=4,
                     h=1, rI=144, rE=144, rY=10000):
    
    plt.rcParams["figure.figsize"] = (8,3)

    color_vals = np.zeros([P-e+1,31], dtype=int)
    for a in range(0,31):
        for i in range(0, P-e+1):
            predictor = StabilityPredictor(i, e, N, P, R, g/52, a/100/52, h/52, rI/52, rE/52, rY/52)
            _, status, _ = predictor.run()
            c_value = None
            if status == 'PASS':
                c_value = 1
            elif status == 'FAIL_SS':
                c_value = 2
            elif status == 'FAIL_IC':
                c_value = 3
            else:
                c_value = 4
            color_vals[i,a] = c_value

    values = [0,1,2,3,4]
    cmp = ListedColormap(['white','limegreen', 'lightslategray', 'chocolate', 'royalblue'])
    im = plt.imshow(color_vals, origin='lower', cmap=cmp, vmin=0, vmax=4)
    colors = [ cmp(value) for value in values]

    xP = np.array(P)
    patches = [ mpatches.Patch(color=colors[1], label="Success"),
            mpatches.Patch(color=colors[2], label="Steady State Failure"),
            mpatches.Patch(color=colors[3], label="Initial Condition Failure"),
            mpatches.Patch(color=colors[4], label="Mentor-Limited Crash")]
    plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.title('Stability Based on Initial Inexperienced Pilots and Attrition Rate')
    plt.xlabel('a')
    plt.ylabel('I0')
    plt.xticks(np.arange(-.5, P+0.5, 1), minor=True)
    plt.yticks(np.arange(-.5, P-e+0.5, 1), minor=True)
    plt.grid(which='minor')

widgets.interact_manual(plot_predictions, N=widgets.IntSlider(value=25, min=0, max=100),
               P=widgets.IntSlider(value=30, min=0, max=100),
               R=widgets.IntSlider(value=250, min=0, max=1000),
               g=widgets.IntSlider(value=4, min=0, max=100),
               h=widgets.IntSlider(value=1, min=0, max=100),
               rI=widgets.IntSlider(value=144, min=0, max=1000),
               rE=widgets.IntSlider(value=144, min=0, max=1000),
               rY=widgets.IntSlider(value=10000, min=0, max=100000),
               E=widgets.IntSlider(value=10, min=0, max=100))
pass

interactive(children=(IntSlider(value=10, description='e', max=30, min=-10), IntSlider(value=25, description='…

### Predicted Stability of the System Depending on Initial Experienced and Non-Operational Positions

In [106]:
%matplotlib inline
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap

def plot_predictions(i=10, a=0.07, P=30, R=250, g=4,
                     h=1, rI=144, rE=144, rY=10000):
    
    plt.rcParams["figure.figsize"] = (8,3)

    color_vals = np.zeros([P-i+1,P+1], dtype=int)
    for N in range(0,P+1):
        for e in range(0, P-i+1):
            predictor = StabilityPredictor(i, e, N, P, R, g/52, a/52, h/52, rI/52, rE/52, rY/52)
            _, status, _ = predictor.run()
            c_value = None
            if status == 'PASS':
                c_value = 1
            elif status == 'FAIL_SS':
                c_value = 2
            elif status == 'FAIL_IC':
                c_value = 3
            else:
                c_value = 4
            color_vals[e,N] = c_value

    values = [0,1,2,3,4]
    cmp = ListedColormap(['white','limegreen', 'lightslategray', 'chocolate', 'royalblue'])
    im = plt.imshow(color_vals, origin='lower', cmap=cmp, vmin=0, vmax=4)
    colors = [ cmp(value) for value in values]

    xP = np.array(P)
    patches = [ mpatches.Patch(color=colors[1], label="Success"),
            mpatches.Patch(color=colors[2], label="Steady State Failure"),
            mpatches.Patch(color=colors[3], label="Initial Condition Failure"),
            mpatches.Patch(color=colors[4], label="Mentor-Limited Crash")]
    plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.title('Stability Based on Initial Experienced Pilots and Non-Operational Positions')
    plt.xlabel('N')
    plt.ylabel('E0')
    plt.xticks(np.arange(-.5, P+0.5, 1), minor=True)
    plt.yticks(np.arange(-.5, P-i+0.5, 1), minor=True)
    plt.grid(which='minor')

widgets.interact_manual(plot_predictions, a=widgets.FloatSlider(value=0.07, min=0, max=1, step=0.01),
               P=widgets.IntSlider(value=30, min=0, max=100),
               R=widgets.IntSlider(value=250, min=0, max=1000),
               g=widgets.IntSlider(value=4, min=0, max=100),
               h=widgets.IntSlider(value=1, min=0, max=100),
               rI=widgets.IntSlider(value=144, min=0, max=1000),
               rE=widgets.IntSlider(value=144, min=0, max=1000),
               rY=widgets.IntSlider(value=10000, min=0, max=100000),
               I=widgets.IntSlider(value=10, min=0, max=10))
pass

interactive(children=(IntSlider(value=10, description='i', max=30, min=-10), FloatSlider(value=0.07, descripti…

### Predicted Stability of the System Depending on Initial Inexperienced and Non-Operational Positions

In [108]:
%matplotlib inline
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap

def plot_predictions(e=10, a=0.07, P=30, R=250, g=4,
                     h=1, rI=144, rE=144, rY=10000):
    
    plt.rcParams["figure.figsize"] = (8,3)

    color_vals = np.zeros([P-e+1,P+1], dtype=int)
    for N in range(0,P+1):
        for i in range(0, P-e+1):
            predictor = StabilityPredictor(i, e, N, P, R, g/52, a/52, h/52, rI/52, rE/52, rY/52)
            _, status, _ = predictor.run()
            c_value = None
            if status == 'PASS':
                c_value = 1
            elif status == 'FAIL_SS':
                c_value = 2
            elif status == 'FAIL_IC':
                c_value = 3
            else:
                c_value = 4
            color_vals[i,N] = c_value

    values = [0,1,2,3,4]
    cmp = ListedColormap(['white','limegreen', 'lightslategray', 'chocolate', 'royalblue'])
    im = plt.imshow(color_vals, origin='lower', cmap=cmp, vmin=0, vmax=4)
    colors = [ cmp(value) for value in values]

    xP = np.array(P)
    patches = [ mpatches.Patch(color=colors[1], label="Success"),
            mpatches.Patch(color=colors[2], label="Steady State Failure"),
            mpatches.Patch(color=colors[3], label="Initial Condition Failure"),
            mpatches.Patch(color=colors[4], label="Mentor-Limited Crash")]
    plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.title('Stability Based on Initial Inexperienced Pilots and Non-Operational Positions')
    plt.xlabel('N')
    plt.ylabel('I0')
    plt.xticks(np.arange(-.5, P+0.5, 1), minor=True)
    plt.yticks(np.arange(-.5, P-e+0.5, 1), minor=True)
    plt.grid(which='minor')

widgets.interact_manual(plot_predictions, a=widgets.FloatSlider(value=0.07, min=0, max=1, step=0.01),
               P=widgets.IntSlider(value=30, min=0, max=100),
               R=widgets.IntSlider(value=250, min=0, max=1000),
               g=widgets.IntSlider(value=4, min=0, max=100),
               h=widgets.IntSlider(value=1, min=0, max=100),
               rI=widgets.IntSlider(value=144, min=0, max=1000),
               rE=widgets.IntSlider(value=144, min=0, max=1000),
               rY=widgets.IntSlider(value=10000, min=0, max=100000),
               e=widgets.IntSlider(value=10, min=0, max=100))
pass

interactive(children=(IntSlider(value=10, description='e'), FloatSlider(value=0.07, description='a', max=1.0, …

### Predicted Stability of the System Depending on Attrition Rate and Non-Operational Positions

In [109]:
%matplotlib inline
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt

import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap

def plot_predictions(e=10, i=10, P=30, R=250, g=4,
                     h=1, rI=144, rE=144, rY=10000):
    
    plt.rcParams["figure.figsize"] = (8,5)

    color_vals = np.zeros([31,P+1], dtype=int)
    for N in range(0,P+1):
        for a in range(0, 31):
            predictor = StabilityPredictor(i, e, N, P, R, g/52, a/100/52, h/52, rI/52, rE/52, rY/52)
            _, status, _ = predictor.run()
            c_value = None
            if status == 'PASS':
                c_value = 1
            elif status == 'FAIL_SS':
                c_value = 2
            elif status == 'FAIL_IC':
                c_value = 3
            else:
                c_value = 4
            color_vals[a,N] = c_value

    values = [0,1,2,3,4]
    cmp = ListedColormap(['white','limegreen', 'lightslategray', 'chocolate', 'royalblue'])
    im = plt.imshow(color_vals, origin='lower', cmap=cmp, vmin=0, vmax=4)
    colors = [ cmp(value) for value in values]

    xP = np.array(P)
    patches = [ mpatches.Patch(color=colors[1], label="Success"),
            mpatches.Patch(color=colors[2], label="Steady State Failure"),
            mpatches.Patch(color=colors[3], label="Initial Condition Failure"),
            mpatches.Patch(color=colors[4], label="Mentor-Limited Crash")]
    plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.title('Stability Based on Non-Operational Positions and Attrition Rate')
    plt.xlabel('N')
    plt.ylabel('a')
    plt.xticks(np.arange(-.5, P+0.5, 1), minor=True)
    plt.yticks(np.arange(-.5, 30.5, 1), minor=True)
    plt.grid(which='minor')
  
widgets.interact_manual(plot_predictions, i=widgets.IntSlider(value=10, min=0, max=100),
               P=widgets.IntSlider(value=30, min=0, max=100),
               R=widgets.IntSlider(value=250, min=0, max=1000),
               g=widgets.IntSlider(value=4, min=0, max=100),
               h=widgets.IntSlider(value=1, min=0, max=100),
               rI=widgets.IntSlider(value=144, min=0, max=1000),
               rE=widgets.IntSlider(value=144, min=0, max=1000),
               rY=widgets.IntSlider(value=10000, min=0, max=100000),
               e=widgets.IntSlider(value=10, min=0, max=100))
pass

interactive(children=(IntSlider(value=10, description='e'), IntSlider(value=10, description='i'), IntSlider(va…